# Converting Data for Visualization

Although we've managed to extract a few examples of both dabs and tposes, it's now time to figure out what our data looks like. 

The easiest way to manipulate and visualize data in Python is via tools like Pandas and Seaborn. 

But first, we'll need to convert our numpy raw arrays into something that's a bit more readable. So let's do that by converting them into labeled CSV files.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dabDataset = np.load('data/dabs.npy')
tposeDataset = np.load('data/tposes.npy')
otherDataset = np.load('data/other.npy')

In [3]:
dabDataset[0]

array([[5.8832416e+02, 2.9433704e+02, 7.2265184e-01],
       [5.8239331e+02, 3.5126093e+02, 8.0205584e-01],
       [5.0984329e+02, 3.4919385e+02, 7.5316119e-01],
       [4.1784265e+02, 3.1985785e+02, 8.1164622e-01],
       [3.6101605e+02, 2.9243521e+02, 8.0296052e-01],
       [6.5091376e+02, 3.6097537e+02, 6.4161348e-01],
       [6.3724268e+02, 2.7274924e+02, 7.8188539e-01],
       [4.9614203e+02, 2.4154723e+02, 8.3243752e-01],
       [5.4315808e+02, 6.4114813e+02, 4.4807938e-01],
       [4.8636816e+02, 6.2938318e+02, 3.6906898e-01],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [6.0191382e+02, 6.4702966e+02, 3.8946095e-01],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [5.7648334e+02, 2.7475522e+02, 6.1822432e-01],
       [6.0389270e+02, 2.8454663e+02, 4.1854110e-01],
       [5.5686536e+02, 2.6891223e+02, 2.7014270e-01],
       [6.1959991e+02, 2.924

In [4]:
dabDataset[0].shape

(25, 3)

# Adding our Labels

Our labels come from the [BODY_25 Pose Output format](https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/output.md#pose-output-format-body_25) available at the repo. 

We can tell because when we looked at each of our poses, we saw a `dataset[0].shape` of 25. This matches the number of labels below.

In [5]:
labels = ["Nose", "Neck", "RShoulder", "RElbow", "RWrist", "LShoulder", "LElbow",
 "LWrist", "MidHip", "RHip", "RKnee", "RAnkle", "LHip", "LKnee", "LAnkle",
 "REye", "LEye", "REar", "LEar", "LBigToe", "LSmallToe", "LHeel", "RBigToe",
 "RSmallToe", "RHeel", "Background"]

Each of our labels comes as an `X`, `Y`, and `Confidence`. Let's add those labels and flatten this array for our CSV file:

In [6]:
properLabels = []
for label in labels:
    properLabels.append(label + 'X')
    properLabels.append(label + 'Y')
    properLabels.append(label + 'Confidence')

In [7]:
import csv

with open('data/dabs.csv', 'w+') as dabcsv:
    dabwriter = csv.writer(dabcsv, delimiter=',')
    dabwriter.writerow(properLabels)
    for cell in dabDataset:
        dabwriter.writerow(cell.flatten())
        
with open('data/tposes.csv', 'w+') as tposecsv:
    tposewriter = csv.writer(tposecsv, delimiter=',')
    tposewriter.writerow(properLabels)
    for cell in tposeDataset:
        tposewriter.writerow(cell.flatten())
        
with open('data/other.csv', 'w+') as othercsv:
    otherwriter = csv.writer(othercsv, delimiter=',')
    otherwriter.writerow(properLabels)
    for cell in otherDataset:
        otherwriter.writerow(cell.flatten())

## Sanity Checking our Data

We can now open up our CSV files and see what they look like. How many samples did we collect? Is it enough? 

Once we check, we can hop on to the next step, bringing all the data into a single format and file for training.

## Creating a Labeled Dataset for Training and Testing

Now that we've got our data (mostly) sorted out, we need to convert it into a set. 

We'll use `0` for `other` poses, `1` for `dabs`, and `2` for `tposes`.



In [8]:
labels = np.zeros(len(otherDataset))
labels = np.append(labels, np.full((len(dabDataset)), 1))
labels = np.append(labels, np.full((len(tposeDataset)), 2))
print(labels)
print("%i total examples for training." % len(labels))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2.]
56 total examples for training.


In [9]:
dataset = np.append(otherDataset, dabDataset, axis=0)
dataset = np.append(dataset, tposeDataset, axis=0)
print(dataset)

[[[488.3213     147.51425      0.83340967]
  [494.22372    284.5734       0.8012297 ]
  [386.4863     270.83716      0.66853976]
  ...
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]]

 [[515.7737     112.18818      0.83487195]
  [478.48004    274.7029       0.8005627 ]
  [368.77948    257.2105       0.6782713 ]
  ...
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]]

 [[547.1316     112.15151      0.79948723]
  [464.79065    268.88403      0.73338044]
  [360.98135    243.43745      0.62600124]
  ...
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]]

 ...

 [[509.97504    257.06958      0.892523  ]
  [460.9663     351.17117      0.7867987 ]
  [372.75305    333.54434      0.6111988 ]
  ...
  [  0.           0.           0.        ]
  [  0.           

In [10]:
dataset.shape

(56, 25, 3)

In [11]:
dataset[:,:,1] / 720

array([[0.20488091, 0.3952408 , 0.3761627 , ..., 0.        , 0.        ,
        0.        ],
       [0.15581691, 0.38153183, 0.35723683, ..., 0.        , 0.        ,
        0.        ],
       [0.155766  , 0.37345004, 0.3381076 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.3570411 , 0.48773775, 0.46325603, ..., 0.        , 0.        ,
        0.        ],
       [0.39792854, 0.5203946 , 0.49046776, ..., 0.        , 0.        ,
        0.        ],
       [0.3734503 , 0.51507103, 0.4904025 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [12]:
# now, let's shuffle labels and the array, the same way
from sklearn.utils import shuffle
X, y = shuffle(dataset, labels)
# now let's label them for 'one hot'
from keras.utils.np_utils import to_categorical
y = to_categorical(y, 3)
print(y.shape[1])

Using TensorFlow backend.


3


In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD

In [14]:
X = X.reshape(56, 75)
dataset[0]

array([[4.88321289e+02, 1.47514252e+02, 8.33409667e-01],
       [4.94223724e+02, 2.84573395e+02, 8.01229715e-01],
       [3.86486298e+02, 2.70837158e+02, 6.68539762e-01],
       [3.37498718e+02, 4.31440033e+02, 8.06459844e-01],
       [2.76727325e+02, 5.92155334e+02, 6.95721209e-01],
       [6.01926575e+02, 2.96297577e+02, 6.77372575e-01],
       [6.17621460e+02, 4.47154175e+02, 8.15527081e-01],
       [6.33207092e+02, 6.13721497e+02, 7.50288665e-01],
       [4.49166534e+02, 6.23515259e+02, 3.33123893e-01],
       [3.68768433e+02, 6.15664124e+02, 2.96909660e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.29464417e+02, 6.35266663e+02, 3.00662249e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.72626495e+02, 1.25848732e+02, 7.93599010e-01],
       [5.09897217e+02, 1.25915306e+02, 8.75047982e-01],
       [4.47158661e+02, 1.31820

In [15]:
opt = SGD(lr=0.005)
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(75,)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer=opt, #'Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X, y, epochs=200,batch_size=50)

Epoch 1/200
56/56 [==============================] - 0s 2ms/step - loss: 11.9271 - acc: 0.2500
Epoch 2/200
56/56 [==============================] - 0s 51us/step - loss: 8.9225 - acc: 0.4464
Epoch 3/200
56/56 [==============================] - 0s 51us/step - loss: 8.6347 - acc: 0.4643
Epoch 4/200
56/56 [==============================] - 0s 49us/step - loss: 8.6347 - acc: 0.4643
Epoch 5/200
56/56 [==============================] - 0s 57us/step - loss: 8.6347 - acc: 0.4643
Epoch 6/200
56/56 [==============================] - 0s 73us/step - loss: 8.6347 - acc: 0.4643
Epoch 7/200
56/56 [==============================] - 0s 71us/step - loss: 8.6347 - acc: 0.4643
Epoch 8/200
56/56 [==============================] - 0s 53us/step - loss: 8.6347 - acc: 0.4643
Epoch 9/200
56/56 [==============================] - 0s 44us/step - loss: 8.9225 - acc: 0.4464
Epoch 10/200
56/56 [==============================] - 0s 45us/step - loss: 8.6347 - acc: 0.4643
Epoch 11/200
56/56 [=============================

56/56 [==============================] - 0s 47us/step - loss: 8.6347 - acc: 0.4643
Epoch 87/200
56/56 [==============================] - 0s 57us/step - loss: 8.6347 - acc: 0.4643
Epoch 88/200
56/56 [==============================] - 0s 43us/step - loss: 8.6347 - acc: 0.4643
Epoch 89/200
56/56 [==============================] - 0s 40us/step - loss: 8.6347 - acc: 0.4643
Epoch 90/200
56/56 [==============================] - 0s 57us/step - loss: 8.6347 - acc: 0.4643
Epoch 91/200
56/56 [==============================] - 0s 63us/step - loss: 8.6347 - acc: 0.4643
Epoch 92/200
56/56 [==============================] - 0s 80us/step - loss: 8.6347 - acc: 0.4643
Epoch 93/200
56/56 [==============================] - 0s 46us/step - loss: 8.6347 - acc: 0.4643
Epoch 94/200
56/56 [==============================] - 0s 82us/step - loss: 8.6347 - acc: 0.4643
Epoch 95/200
56/56 [==============================] - 0s 49us/step - loss: 8.6347 - acc: 0.4643
Epoch 96/200
56/56 [==============================] -

56/56 [==============================] - 0s 50us/step - loss: 8.6347 - acc: 0.4643
Epoch 171/200
56/56 [==============================] - 0s 55us/step - loss: 8.6347 - acc: 0.4643
Epoch 172/200
56/56 [==============================] - ETA: 0s - loss: 8.7038 - acc: 0.460 - 0s 55us/step - loss: 8.6347 - acc: 0.4643
Epoch 173/200
56/56 [==============================] - 0s 48us/step - loss: 8.6347 - acc: 0.4643
Epoch 174/200
56/56 [==============================] - 0s 56us/step - loss: 8.6347 - acc: 0.4643
Epoch 175/200
56/56 [==============================] - 0s 53us/step - loss: 8.6347 - acc: 0.4643
Epoch 176/200
56/56 [==============================] - 0s 54us/step - loss: 8.6347 - acc: 0.4643
Epoch 177/200
56/56 [==============================] - 0s 49us/step - loss: 8.6347 - acc: 0.4643
Epoch 178/200
56/56 [==============================] - 0s 54us/step - loss: 8.6347 - acc: 0.4643
Epoch 179/200
56/56 [==============================] - 0s 51us/step - loss: 8.6347 - acc: 0.4643
Epoch 

# Cleaning up data further

Looking at our accuracy, it looks like we need to better prepare and nomalize our data. 

Or maybe we need to try a different optimizer. Here, I've tried both `SGD` and `Adam`, and saw no improvement.

In [16]:
# let's fit x and y to 0 - 1, get rid of confidence and try again
X, y = shuffle(dataset, labels)
y = to_categorical(y, 3)
print(X.shape)
X[:,:,0] = X[:,:,0] / 1280 # I think the dimensions are 1280 x 720 ?
X[:,:,1] = X[:,:,1] / 720  # let's see?
X = X[:,:,1:]
print(X.shape)
X = X.reshape(56, 50)      # we got rid of confidence percentage
print(X.shape)

(56, 25, 3)
(56, 25, 2)
(56, 50)


In [18]:
opt = SGD(lr=0.01)
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(50,)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X, y, epochs=2000,batch_size=25)

Epoch 1/2000
56/56 [==============================] - 0s 3ms/step - loss: 1.1297 - acc: 0.3393
Epoch 2/2000
56/56 [==============================] - 0s 78us/step - loss: 1.1530 - acc: 0.3214
Epoch 3/2000
56/56 [==============================] - 0s 87us/step - loss: 1.0687 - acc: 0.4286
Epoch 4/2000
56/56 [==============================] - 0s 81us/step - loss: 1.1398 - acc: 0.3750
Epoch 5/2000
56/56 [==============================] - 0s 70us/step - loss: 1.0945 - acc: 0.4643
Epoch 6/2000
56/56 [==============================] - 0s 75us/step - loss: 1.0261 - acc: 0.5000
Epoch 7/2000
56/56 [==============================] - 0s 67us/step - loss: 1.0078 - acc: 0.5357
Epoch 8/2000
56/56 [==============================] - 0s 58us/step - loss: 1.0329 - acc: 0.4286
Epoch 9/2000
56/56 [==============================] - 0s 67us/step - loss: 1.0742 - acc: 0.4286
Epoch 10/2000
56/56 [==============================] - 0s 83us/step - loss: 0.9895 - acc: 0.5536
Epoch 11/2000
56/56 [===================

56/56 [==============================] - 0s 74us/step - loss: 0.3875 - acc: 0.8571
Epoch 86/2000
56/56 [==============================] - 0s 66us/step - loss: 0.3564 - acc: 0.8036
Epoch 87/2000
56/56 [==============================] - 0s 77us/step - loss: 0.3461 - acc: 0.8393
Epoch 88/2000
56/56 [==============================] - 0s 103us/step - loss: 0.4311 - acc: 0.7500
Epoch 89/2000
56/56 [==============================] - 0s 71us/step - loss: 0.4129 - acc: 0.8393
Epoch 90/2000
56/56 [==============================] - 0s 69us/step - loss: 0.3979 - acc: 0.8214
Epoch 91/2000
56/56 [==============================] - 0s 63us/step - loss: 0.3781 - acc: 0.8214
Epoch 92/2000
56/56 [==============================] - 0s 98us/step - loss: 0.3670 - acc: 0.8036
Epoch 93/2000
56/56 [==============================] - 0s 78us/step - loss: 0.3997 - acc: 0.8571
Epoch 94/2000
56/56 [==============================] - 0s 70us/step - loss: 0.4295 - acc: 0.7857
Epoch 95/2000
56/56 [======================

56/56 [==============================] - 0s 60us/step - loss: 0.2974 - acc: 0.8036
Epoch 169/2000
56/56 [==============================] - 0s 63us/step - loss: 0.2748 - acc: 0.8571
Epoch 170/2000
56/56 [==============================] - 0s 63us/step - loss: 0.2940 - acc: 0.8036
Epoch 171/2000
56/56 [==============================] - 0s 113us/step - loss: 0.2589 - acc: 0.8929
Epoch 172/2000
56/56 [==============================] - 0s 63us/step - loss: 0.2882 - acc: 0.8393
Epoch 173/2000
56/56 [==============================] - 0s 73us/step - loss: 0.2578 - acc: 0.8929
Epoch 174/2000
56/56 [==============================] - 0s 83us/step - loss: 0.2777 - acc: 0.8929
Epoch 175/2000
56/56 [==============================] - 0s 86us/step - loss: 0.2767 - acc: 0.8393
Epoch 176/2000
56/56 [==============================] - 0s 75us/step - loss: 0.2462 - acc: 0.8750
Epoch 177/2000
56/56 [==============================] - 0s 81us/step - loss: 0.2728 - acc: 0.9107
Epoch 178/2000
56/56 [============

56/56 [==============================] - 0s 75us/step - loss: 0.2138 - acc: 0.8929
Epoch 252/2000
56/56 [==============================] - 0s 78us/step - loss: 0.2023 - acc: 0.9286
Epoch 253/2000
56/56 [==============================] - 0s 65us/step - loss: 0.2232 - acc: 0.8750
Epoch 254/2000
56/56 [==============================] - 0s 68us/step - loss: 0.1887 - acc: 0.9107
Epoch 255/2000
56/56 [==============================] - 0s 66us/step - loss: 0.1920 - acc: 0.8929
Epoch 256/2000
56/56 [==============================] - 0s 75us/step - loss: 0.2132 - acc: 0.8750
Epoch 257/2000
56/56 [==============================] - 0s 65us/step - loss: 0.2348 - acc: 0.8929
Epoch 258/2000
56/56 [==============================] - 0s 75us/step - loss: 0.1659 - acc: 0.9286
Epoch 259/2000
56/56 [==============================] - 0s 60us/step - loss: 0.1770 - acc: 0.9107
Epoch 260/2000
56/56 [==============================] - 0s 66us/step - loss: 0.2178 - acc: 0.9107
Epoch 261/2000
56/56 [=============

56/56 [==============================] - 0s 59us/step - loss: 0.1654 - acc: 0.8750
Epoch 335/2000
56/56 [==============================] - 0s 59us/step - loss: 0.1338 - acc: 0.9464
Epoch 336/2000
56/56 [==============================] - 0s 68us/step - loss: 0.1442 - acc: 0.9643
Epoch 337/2000
56/56 [==============================] - 0s 64us/step - loss: 0.1235 - acc: 0.9464
Epoch 338/2000
56/56 [==============================] - 0s 58us/step - loss: 0.1354 - acc: 0.9286
Epoch 339/2000
56/56 [==============================] - 0s 57us/step - loss: 0.1642 - acc: 0.9107
Epoch 340/2000
56/56 [==============================] - 0s 79us/step - loss: 0.1456 - acc: 0.9643
Epoch 341/2000
56/56 [==============================] - 0s 88us/step - loss: 0.1587 - acc: 0.9286
Epoch 342/2000
56/56 [==============================] - 0s 59us/step - loss: 0.1538 - acc: 0.9107
Epoch 343/2000
56/56 [==============================] - 0s 93us/step - loss: 0.1631 - acc: 0.9464
Epoch 344/2000
56/56 [=============

...

56/56 [==============================] - 0s 100us/step - loss: 0.0527 - acc: 0.9821
Epoch 1655/2000
56/56 [==============================] - 0s 71us/step - loss: 0.0393 - acc: 0.9643
Epoch 1656/2000
56/56 [==============================] - 0s 67us/step - loss: 0.0452 - acc: 1.0000
Epoch 1657/2000
56/56 [==============================] - 0s 85us/step - loss: 0.0442 - acc: 1.0000
Epoch 1658/2000
56/56 [==============================] - 0s 71us/step - loss: 0.0370 - acc: 0.9821
Epoch 1659/2000
56/56 [==============================] - 0s 73us/step - loss: 0.0533 - acc: 0.9821
Epoch 1660/2000
56/56 [==============================] - 0s 71us/step - loss: 0.0358 - acc: 0.9821
Epoch 1661/2000
56/56 [==============================] - 0s 83us/step - loss: 0.0733 - acc: 0.9464
Epoch 1662/2000
56/56 [==============================] - 0s 84us/step - loss: 0.0427 - acc: 1.0000
Epoch 1663/2000
56/56 [==============================] - 0s 65us/step - loss: 0.0513 - acc: 0.9821
Epoch 1664/2000
56/56 [==

56/56 [==============================] - 0s 78us/step - loss: 0.0395 - acc: 1.0000
Epoch 1737/2000
56/56 [==============================] - 0s 151us/step - loss: 0.0523 - acc: 0.9821
Epoch 1738/2000
56/56 [==============================] - 0s 100us/step - loss: 0.0289 - acc: 0.9821
Epoch 1739/2000
56/56 [==============================] - 0s 88us/step - loss: 0.0624 - acc: 0.9821
Epoch 1740/2000
56/56 [==============================] - 0s 74us/step - loss: 0.0186 - acc: 1.0000
Epoch 1741/2000
56/56 [==============================] - 0s 69us/step - loss: 0.0522 - acc: 0.9643
Epoch 1742/2000
56/56 [==============================] - 0s 91us/step - loss: 0.0518 - acc: 0.9821
Epoch 1743/2000
56/56 [==============================] - 0s 83us/step - loss: 0.0813 - acc: 0.9821
Epoch 1744/2000
56/56 [==============================] - 0s 75us/step - loss: 0.0395 - acc: 0.9643
Epoch 1745/2000
56/56 [==============================] - 0s 84us/step - loss: 0.0254 - acc: 1.0000
Epoch 1746/2000
56/56 [=

56/56 [==============================] - 0s 113us/step - loss: 0.0212 - acc: 1.0000
Epoch 1819/2000
56/56 [==============================] - 0s 83us/step - loss: 0.0587 - acc: 0.9643
Epoch 1820/2000
56/56 [==============================] - 0s 61us/step - loss: 0.0373 - acc: 1.0000
Epoch 1821/2000
56/56 [==============================] - 0s 77us/step - loss: 0.0880 - acc: 0.9821
Epoch 1822/2000
56/56 [==============================] - 0s 82us/step - loss: 0.0817 - acc: 0.9821
Epoch 1823/2000
56/56 [==============================] - 0s 82us/step - loss: 0.0998 - acc: 0.9643
Epoch 1824/2000
56/56 [==============================] - 0s 77us/step - loss: 0.0315 - acc: 0.9821
Epoch 1825/2000
56/56 [==============================] - 0s 78us/step - loss: 0.0559 - acc: 0.9643
Epoch 1826/2000
56/56 [==============================] - 0s 68us/step - loss: 0.0193 - acc: 1.0000
Epoch 1827/2000
56/56 [==============================] - 0s 72us/step - loss: 0.0527 - acc: 0.9821
Epoch 1828/2000
56/56 [==

56/56 [==============================] - 0s 94us/step - loss: 0.0366 - acc: 1.0000
Epoch 1901/2000
56/56 [==============================] - 0s 65us/step - loss: 0.0291 - acc: 0.9821
Epoch 1902/2000
56/56 [==============================] - 0s 63us/step - loss: 0.0118 - acc: 1.0000
Epoch 1903/2000
56/56 [==============================] - 0s 72us/step - loss: 0.0156 - acc: 1.0000
Epoch 1904/2000
56/56 [==============================] - 0s 76us/step - loss: 0.0316 - acc: 1.0000
Epoch 1905/2000
56/56 [==============================] - 0s 69us/step - loss: 0.0293 - acc: 0.9821
Epoch 1906/2000
56/56 [==============================] - 0s 67us/step - loss: 0.0547 - acc: 0.9821
Epoch 1907/2000
56/56 [==============================] - 0s 74us/step - loss: 0.0185 - acc: 1.0000
Epoch 1908/2000
56/56 [==============================] - 0s 85us/step - loss: 0.0328 - acc: 0.9821
Epoch 1909/2000
56/56 [==============================] - 0s 63us/step - loss: 0.0196 - acc: 1.0000
Epoch 1910/2000
56/56 [===

56/56 [==============================] - 0s 89us/step - loss: 0.0910 - acc: 0.9643
Epoch 1983/2000
56/56 [==============================] - 0s 101us/step - loss: 0.2367 - acc: 0.8750
Epoch 1984/2000
56/56 [==============================] - 0s 73us/step - loss: 0.0872 - acc: 0.9464
Epoch 1985/2000
56/56 [==============================] - 0s 71us/step - loss: 0.1004 - acc: 0.9643
Epoch 1986/2000
56/56 [==============================] - 0s 73us/step - loss: 0.0493 - acc: 1.0000
Epoch 1987/2000
56/56 [==============================] - 0s 68us/step - loss: 0.0396 - acc: 0.9821
Epoch 1988/2000
56/56 [==============================] - 0s 59us/step - loss: 0.0505 - acc: 0.9821
Epoch 1989/2000
56/56 [==============================] - 0s 100us/step - loss: 0.0851 - acc: 0.9464
Epoch 1990/2000
56/56 [==============================] - 0s 80us/step - loss: 0.0480 - acc: 0.9821
Epoch 1991/2000
56/56 [==============================] - 0s 71us/step - loss: 0.0611 - acc: 0.9821
Epoch 1992/2000
56/56 [=

In [19]:
opt = SGD(lr=0.01)
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(50,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X, y, epochs=5000,batch_size=25)

Epoch 1/5000
56/56 [==============================] - 0s 5ms/step - loss: 1.1564 - acc: 0.3750
Epoch 2/5000
56/56 [==============================] - 0s 62us/step - loss: 1.2458 - acc: 0.4107
Epoch 3/5000
56/56 [==============================] - 0s 61us/step - loss: 1.1773 - acc: 0.3750
Epoch 4/5000
56/56 [==============================] - 0s 66us/step - loss: 1.2714 - acc: 0.3393
Epoch 5/5000
56/56 [==============================] - 0s 68us/step - loss: 1.1987 - acc: 0.3393
Epoch 6/5000
56/56 [==============================] - 0s 61us/step - loss: 1.1685 - acc: 0.3929
Epoch 7/5000
56/56 [==============================] - 0s 67us/step - loss: 1.2117 - acc: 0.4107
Epoch 8/5000
56/56 [==============================] - 0s 70us/step - loss: 1.2385 - acc: 0.3214
Epoch 9/5000
56/56 [==============================] - 0s 68us/step - loss: 1.0942 - acc: 0.3929
Epoch 10/5000
56/56 [==============================] - 0s 61us/step - loss: 1.0694 - acc: 0.4821
Epoch 11/5000
56/56 [===================

56/56 [==============================] - 0s 71us/step - loss: 0.7038 - acc: 0.6786
Epoch 86/5000
56/56 [==============================] - 0s 70us/step - loss: 0.6640 - acc: 0.7321
Epoch 87/5000
56/56 [==============================] - 0s 77us/step - loss: 0.6898 - acc: 0.6429
Epoch 88/5000
56/56 [==============================] - 0s 64us/step - loss: 0.6891 - acc: 0.6964
Epoch 89/5000
56/56 [==============================] - 0s 81us/step - loss: 0.6270 - acc: 0.6964
Epoch 90/5000
56/56 [==============================] - 0s 122us/step - loss: 0.6115 - acc: 0.7321
Epoch 91/5000
56/56 [==============================] - 0s 83us/step - loss: 0.6752 - acc: 0.6964
Epoch 92/5000
56/56 [==============================] - 0s 134us/step - loss: 0.6026 - acc: 0.7500
Epoch 93/5000
56/56 [==============================] - 0s 86us/step - loss: 0.7022 - acc: 0.6429
Epoch 94/5000
56/56 [==============================] - 0s 86us/step - loss: 0.6464 - acc: 0.6964
Epoch 95/5000
56/56 [=====================

56/56 [==============================] - 0s 71us/step - loss: 0.4924 - acc: 0.7857
Epoch 169/5000
56/56 [==============================] - 0s 89us/step - loss: 0.4936 - acc: 0.7143
Epoch 170/5000
56/56 [==============================] - 0s 64us/step - loss: 0.4480 - acc: 0.6964
Epoch 171/5000
56/56 [==============================] - 0s 99us/step - loss: 0.4726 - acc: 0.7500
Epoch 172/5000
56/56 [==============================] - 0s 76us/step - loss: 0.5608 - acc: 0.7321
Epoch 173/5000
56/56 [==============================] - 0s 66us/step - loss: 0.4653 - acc: 0.6786
Epoch 174/5000
56/56 [==============================] - 0s 84us/step - loss: 0.4593 - acc: 0.7857
Epoch 175/5000
56/56 [==============================] - 0s 78us/step - loss: 0.4087 - acc: 0.7679
Epoch 176/5000
56/56 [==============================] - 0s 68us/step - loss: 0.4109 - acc: 0.7321
Epoch 177/5000
56/56 [==============================] - 0s 57us/step - loss: 0.4224 - acc: 0.6964
Epoch 178/5000
56/56 [=============

56/56 [==============================] - 0s 73us/step - loss: 0.4321 - acc: 0.7679
Epoch 252/5000
56/56 [==============================] - 0s 74us/step - loss: 0.4106 - acc: 0.6964
Epoch 253/5000
56/56 [==============================] - 0s 102us/step - loss: 0.3728 - acc: 0.8393
Epoch 254/5000
56/56 [==============================] - 0s 79us/step - loss: 0.3862 - acc: 0.8214
Epoch 255/5000
56/56 [==============================] - 0s 70us/step - loss: 0.3883 - acc: 0.7500
Epoch 256/5000
56/56 [==============================] - 0s 84us/step - loss: 0.3928 - acc: 0.7857
Epoch 257/5000
56/56 [==============================] - 0s 66us/step - loss: 0.3541 - acc: 0.8393
Epoch 258/5000
56/56 [==============================] - 0s 64us/step - loss: 0.3836 - acc: 0.7857
Epoch 259/5000
56/56 [==============================] - 0s 66us/step - loss: 0.3872 - acc: 0.8214
Epoch 260/5000
56/56 [==============================] - 0s 98us/step - loss: 0.4140 - acc: 0.7500
Epoch 261/5000
56/56 [============

56/56 [==============================] - 0s 79us/step - loss: 0.3462 - acc: 0.8750
Epoch 335/5000
56/56 [==============================] - 0s 67us/step - loss: 0.3362 - acc: 0.8214
Epoch 336/5000
56/56 [==============================] - 0s 91us/step - loss: 0.3520 - acc: 0.8214
Epoch 337/5000
56/56 [==============================] - 0s 80us/step - loss: 0.3502 - acc: 0.8036
Epoch 338/5000
56/56 [==============================] - 0s 84us/step - loss: 0.3702 - acc: 0.7679
Epoch 339/5000
56/56 [==============================] - 0s 85us/step - loss: 0.3083 - acc: 0.8393
Epoch 340/5000
56/56 [==============================] - 0s 106us/step - loss: 0.3724 - acc: 0.8214
Epoch 341/5000
56/56 [==============================] - 0s 78us/step - loss: 0.3395 - acc: 0.8571
Epoch 342/5000
56/56 [==============================] - 0s 87us/step - loss: 0.3506 - acc: 0.8214
Epoch 343/5000
56/56 [==============================] - 0s 99us/step - loss: 0.4291 - acc: 0.7500
Epoch 344/5000
56/56 [============

...

56/56 [==============================] - 0s 83us/step - loss: 0.1314 - acc: 0.9286
Epoch 4606/5000
56/56 [==============================] - 0s 84us/step - loss: 0.1796 - acc: 0.9286
Epoch 4607/5000
56/56 [==============================] - 0s 89us/step - loss: 0.1574 - acc: 0.9107
Epoch 4608/5000
56/56 [==============================] - 0s 84us/step - loss: 0.2851 - acc: 0.8750
Epoch 4609/5000
56/56 [==============================] - 0s 86us/step - loss: 0.1714 - acc: 0.9107
Epoch 4610/5000
56/56 [==============================] - 0s 84us/step - loss: 0.2180 - acc: 0.8929
Epoch 4611/5000
56/56 [==============================] - 0s 88us/step - loss: 0.1722 - acc: 0.9286
Epoch 4612/5000
56/56 [==============================] - 0s 97us/step - loss: 0.1519 - acc: 0.9643
Epoch 4613/5000
56/56 [==============================] - 0s 85us/step - loss: 0.1695 - acc: 0.9107
Epoch 4614/5000
56/56 [==============================] - 0s 62us/step - loss: 0.1240 - acc: 0.9643
Epoch 4615/5000
56/56 [===

56/56 [==============================] - 0s 79us/step - loss: 0.0960 - acc: 0.9464
Epoch 4688/5000
56/56 [==============================] - 0s 84us/step - loss: 0.0889 - acc: 0.9643
Epoch 4689/5000
56/56 [==============================] - 0s 72us/step - loss: 0.2479 - acc: 0.8571
Epoch 4690/5000
56/56 [==============================] - 0s 90us/step - loss: 0.0817 - acc: 0.9821
Epoch 4691/5000
56/56 [==============================] - 0s 74us/step - loss: 0.0844 - acc: 0.9821
Epoch 4692/5000
56/56 [==============================] - 0s 72us/step - loss: 0.1618 - acc: 0.9643
Epoch 4693/5000
56/56 [==============================] - 0s 75us/step - loss: 0.1376 - acc: 0.9286
Epoch 4694/5000
56/56 [==============================] - 0s 77us/step - loss: 0.2000 - acc: 0.9107
Epoch 4695/5000
56/56 [==============================] - 0s 85us/step - loss: 0.1248 - acc: 0.9464
Epoch 4696/5000
56/56 [==============================] - 0s 72us/step - loss: 0.1200 - acc: 0.9464
Epoch 4697/5000
56/56 [===

56/56 [==============================] - 0s 85us/step - loss: 0.1647 - acc: 0.8750
Epoch 4770/5000
56/56 [==============================] - 0s 94us/step - loss: 0.2026 - acc: 0.8750
Epoch 4771/5000
56/56 [==============================] - 0s 68us/step - loss: 0.0959 - acc: 0.9643
Epoch 4772/5000
56/56 [==============================] - 0s 96us/step - loss: 0.2529 - acc: 0.8750
Epoch 4773/5000
56/56 [==============================] - 0s 81us/step - loss: 0.1055 - acc: 0.9464
Epoch 4774/5000
56/56 [==============================] - 0s 76us/step - loss: 0.1699 - acc: 0.9286
Epoch 4775/5000
56/56 [==============================] - 0s 84us/step - loss: 0.1738 - acc: 0.8929
Epoch 4776/5000
56/56 [==============================] - 0s 68us/step - loss: 0.1499 - acc: 0.9107
Epoch 4777/5000
56/56 [==============================] - 0s 133us/step - loss: 0.2059 - acc: 0.8929
Epoch 4778/5000
56/56 [==============================] - 0s 77us/step - loss: 0.1536 - acc: 0.9107
Epoch 4779/5000
56/56 [==

56/56 [==============================] - 0s 65us/step - loss: 0.1900 - acc: 0.8929
Epoch 4852/5000
56/56 [==============================] - 0s 68us/step - loss: 0.2734 - acc: 0.8750
Epoch 4853/5000
56/56 [==============================] - 0s 96us/step - loss: 0.1934 - acc: 0.8750
Epoch 4854/5000
56/56 [==============================] - 0s 98us/step - loss: 0.2059 - acc: 0.8929
Epoch 4855/5000
56/56 [==============================] - 0s 128us/step - loss: 0.2705 - acc: 0.8571
Epoch 4856/5000
56/56 [==============================] - 0s 78us/step - loss: 0.1990 - acc: 0.8929
Epoch 4857/5000
56/56 [==============================] - 0s 60us/step - loss: 0.1538 - acc: 0.9107
Epoch 4858/5000
56/56 [==============================] - 0s 82us/step - loss: 0.1834 - acc: 0.9107
Epoch 4859/5000
56/56 [==============================] - 0s 69us/step - loss: 0.1930 - acc: 0.8929
Epoch 4860/5000
56/56 [==============================] - 0s 72us/step - loss: 0.2180 - acc: 0.8571
Epoch 4861/5000
56/56 [==

56/56 [==============================] - 0s 70us/step - loss: 0.1908 - acc: 0.9107
Epoch 4934/5000
56/56 [==============================] - 0s 66us/step - loss: 0.2612 - acc: 0.8929
Epoch 4935/5000
56/56 [==============================] - 0s 95us/step - loss: 0.2884 - acc: 0.8929
Epoch 4936/5000
56/56 [==============================] - 0s 81us/step - loss: 0.1918 - acc: 0.9107
Epoch 4937/5000
56/56 [==============================] - 0s 69us/step - loss: 0.1610 - acc: 0.9286
Epoch 4938/5000
56/56 [==============================] - 0s 74us/step - loss: 0.1138 - acc: 0.9464
Epoch 4939/5000
56/56 [==============================] - 0s 91us/step - loss: 0.1403 - acc: 0.9286
Epoch 4940/5000
56/56 [==============================] - 0s 73us/step - loss: 0.1481 - acc: 0.9286
Epoch 4941/5000
56/56 [==============================] - 0s 84us/step - loss: 0.1378 - acc: 0.9643
Epoch 4942/5000
56/56 [==============================] - 0s 65us/step - loss: 0.0853 - acc: 0.9821
Epoch 4943/5000
56/56 [===

# Adding More Data and Beginning Data Augmentation

We've started to improve upon our inital approach. Our accuracy is now approaching 0.76. But now, let's add more examples, and switch between a `train` and `test` dataset.

I've run the data collection program again, and this time doubled the examples we have to train on. Now that we better understand our data, we can speed up it's import and cleaning.

In [20]:
dabDataset = np.load('data/more-dabs.npy')
tposeDataset = np.load('data/more-tposes.npy')
otherDataset = np.load('data/more-other.npy')
labels1 = np.zeros(len(otherDataset))
labels1 = np.append(labels, np.full((len(dabDataset)), 1))
labels1 = np.append(labels, np.full((len(tposeDataset)), 2))
print(labels1)
print("%i total new samples" % len(labels1))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2.]
103 total new samples


In [21]:
dataset1 = np.append(otherDataset, dabDataset, axis=0)
dataset1 = np.append(dataset, tposeDataset, axis=0)
X1, y1 = shuffle(dataset1, labels1)
y1 = to_categorical(y1, 3)
print(X.shape)
X1[:,:,0] = X1[:,:,0] / 1280 # I think the dimensions are 1280 x 720 ?
X1[:,:,1] = X1[:,:,1] / 720  # let's see?
X1 = X1[:,:,1:]
print(X1.shape)
X1 = X1.reshape(103, 50)      # we got rid of confidence percentage
print(X1.shape)

(56, 50)
(103, 25, 2)
(103, 50)


In [22]:
opt = SGD(lr=0.01)
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(50,)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X1, y1, epochs=2000,batch_size=25)

Epoch 1/2000
103/103 [==============================] - 0s 2ms/step - loss: 0.9431 - acc: 0.5631
Epoch 2/2000
103/103 [==============================] - 0s 51us/step - loss: 0.7378 - acc: 0.6408
Epoch 3/2000
103/103 [==============================] - 0s 65us/step - loss: 0.6751 - acc: 0.6505
Epoch 4/2000
103/103 [==============================] - 0s 63us/step - loss: 0.7220 - acc: 0.6408
Epoch 5/2000
103/103 [==============================] - 0s 65us/step - loss: 0.7240 - acc: 0.6117
Epoch 6/2000
103/103 [==============================] - 0s 65us/step - loss: 0.6891 - acc: 0.6117
Epoch 7/2000
103/103 [==============================] - 0s 62us/step - loss: 0.6434 - acc: 0.6408
Epoch 8/2000
103/103 [==============================] - 0s 81us/step - loss: 0.5923 - acc: 0.7184
Epoch 9/2000
103/103 [==============================] - 0s 57us/step - loss: 0.6451 - acc: 0.6408
Epoch 10/2000
103/103 [==============================] - 0s 61us/step - loss: 0.5914 - acc: 0.7184
Epoch 11/2000
103/10

103/103 [==============================] - 0s 72us/step - loss: 0.1885 - acc: 0.9029
Epoch 84/2000
103/103 [==============================] - 0s 60us/step - loss: 0.2133 - acc: 0.8932
Epoch 85/2000
103/103 [==============================] - 0s 55us/step - loss: 0.2348 - acc: 0.9126
Epoch 86/2000
103/103 [==============================] - 0s 61us/step - loss: 0.2382 - acc: 0.8738
Epoch 87/2000
103/103 [==============================] - 0s 59us/step - loss: 0.2390 - acc: 0.8641
Epoch 88/2000
103/103 [==============================] - 0s 65us/step - loss: 0.2071 - acc: 0.9223
Epoch 89/2000
103/103 [==============================] - 0s 62us/step - loss: 0.1979 - acc: 0.9320
Epoch 90/2000
103/103 [==============================] - 0s 49us/step - loss: 0.1900 - acc: 0.9417
Epoch 91/2000
103/103 [==============================] - 0s 62us/step - loss: 0.2195 - acc: 0.8738
Epoch 92/2000
103/103 [==============================] - 0s 53us/step - loss: 0.1733 - acc: 0.9223
Epoch 93/2000
103/103 [=

103/103 [==============================] - 0s 58us/step - loss: 0.1464 - acc: 0.9320
Epoch 166/2000
103/103 [==============================] - 0s 63us/step - loss: 0.1509 - acc: 0.9515
Epoch 167/2000
103/103 [==============================] - 0s 55us/step - loss: 0.1460 - acc: 0.9417
Epoch 168/2000
103/103 [==============================] - 0s 56us/step - loss: 0.1497 - acc: 0.9126
Epoch 169/2000
103/103 [==============================] - 0s 52us/step - loss: 0.1634 - acc: 0.9320
Epoch 170/2000
103/103 [==============================] - 0s 62us/step - loss: 0.1366 - acc: 0.9709
Epoch 171/2000
103/103 [==============================] - 0s 75us/step - loss: 0.1454 - acc: 0.9126
Epoch 172/2000
103/103 [==============================] - 0s 68us/step - loss: 0.1257 - acc: 0.9806
Epoch 173/2000
103/103 [==============================] - 0s 50us/step - loss: 0.1188 - acc: 0.9515
Epoch 174/2000
103/103 [==============================] - 0s 119us/step - loss: 0.1481 - acc: 0.9417
Epoch 175/2000

103/103 [==============================] - 0s 58us/step - loss: 0.1561 - acc: 0.9029
Epoch 247/2000
103/103 [==============================] - 0s 64us/step - loss: 0.1306 - acc: 0.9320
Epoch 248/2000
103/103 [==============================] - 0s 58us/step - loss: 0.1342 - acc: 0.9417
Epoch 249/2000
103/103 [==============================] - 0s 87us/step - loss: 0.1022 - acc: 0.9903
Epoch 250/2000
103/103 [==============================] - 0s 56us/step - loss: 0.1199 - acc: 0.9320
Epoch 251/2000
103/103 [==============================] - 0s 66us/step - loss: 0.1287 - acc: 0.9126
Epoch 252/2000
103/103 [==============================] - 0s 87us/step - loss: 0.1253 - acc: 0.9320
Epoch 253/2000
103/103 [==============================] - 0s 66us/step - loss: 0.0970 - acc: 0.9612
Epoch 254/2000
103/103 [==============================] - 0s 80us/step - loss: 0.1110 - acc: 0.9612
Epoch 255/2000
103/103 [==============================] - 0s 97us/step - loss: 0.1100 - acc: 0.9417
Epoch 256/2000


103/103 [==============================] - 0s 64us/step - loss: 0.1110 - acc: 0.9612
Epoch 329/2000
103/103 [==============================] - 0s 75us/step - loss: 0.1005 - acc: 0.9515
Epoch 330/2000
103/103 [==============================] - 0s 56us/step - loss: 0.0834 - acc: 0.9709
Epoch 331/2000
103/103 [==============================] - 0s 71us/step - loss: 0.0860 - acc: 0.9806
Epoch 332/2000
103/103 [==============================] - 0s 61us/step - loss: 0.0973 - acc: 0.9417
Epoch 333/2000
103/103 [==============================] - 0s 67us/step - loss: 0.1106 - acc: 0.9612
Epoch 334/2000
103/103 [==============================] - 0s 49us/step - loss: 0.0748 - acc: 0.9709
Epoch 335/2000
103/103 [==============================] - 0s 80us/step - loss: 0.1171 - acc: 0.9515
Epoch 336/2000
103/103 [==============================] - 0s 53us/step - loss: 0.1428 - acc: 0.9223
Epoch 337/2000
103/103 [==============================] - 0s 56us/step - loss: 0.1275 - acc: 0.9320
Epoch 338/2000


...

103/103 [==============================] - 0s 60us/step - loss: 0.1268 - acc: 0.9223
Epoch 1628/2000
103/103 [==============================] - 0s 65us/step - loss: 0.0918 - acc: 0.9320
Epoch 1629/2000
103/103 [==============================] - 0s 51us/step - loss: 0.0506 - acc: 0.9806
Epoch 1630/2000
103/103 [==============================] - 0s 74us/step - loss: 0.0763 - acc: 0.9806
Epoch 1631/2000
103/103 [==============================] - 0s 58us/step - loss: 0.0498 - acc: 0.9806
Epoch 1632/2000
103/103 [==============================] - 0s 66us/step - loss: 0.0975 - acc: 0.9515
Epoch 1633/2000
103/103 [==============================] - 0s 57us/step - loss: 0.0764 - acc: 0.9612
Epoch 1634/2000
103/103 [==============================] - 0s 51us/step - loss: 0.0961 - acc: 0.9612
Epoch 1635/2000
103/103 [==============================] - 0s 64us/step - loss: 0.0577 - acc: 0.9806
Epoch 1636/2000
103/103 [==============================] - 0s 53us/step - loss: 0.0768 - acc: 0.9515
Epoch 

103/103 [==============================] - 0s 53us/step - loss: 0.0311 - acc: 0.9903
Epoch 1709/2000
103/103 [==============================] - 0s 53us/step - loss: 0.0546 - acc: 0.9806
Epoch 1710/2000
103/103 [==============================] - 0s 53us/step - loss: 0.0282 - acc: 1.0000
Epoch 1711/2000
103/103 [==============================] - 0s 56us/step - loss: 0.0416 - acc: 0.9806
Epoch 1712/2000
103/103 [==============================] - 0s 66us/step - loss: 0.0298 - acc: 0.9903
Epoch 1713/2000
103/103 [==============================] - 0s 63us/step - loss: 0.0273 - acc: 1.0000
Epoch 1714/2000
103/103 [==============================] - 0s 70us/step - loss: 0.0526 - acc: 0.9709
Epoch 1715/2000
103/103 [==============================] - 0s 58us/step - loss: 0.0446 - acc: 0.9903
Epoch 1716/2000
103/103 [==============================] - 0s 63us/step - loss: 0.0334 - acc: 0.9903
Epoch 1717/2000
103/103 [==============================] - 0s 61us/step - loss: 0.0675 - acc: 0.9709
Epoch 

103/103 [==============================] - 0s 65us/step - loss: 0.0394 - acc: 0.9806
Epoch 1789/2000
103/103 [==============================] - 0s 61us/step - loss: 0.0524 - acc: 0.9806
Epoch 1790/2000
103/103 [==============================] - 0s 62us/step - loss: 0.0690 - acc: 0.9709
Epoch 1791/2000
103/103 [==============================] - 0s 68us/step - loss: 0.0347 - acc: 0.9903
Epoch 1792/2000
103/103 [==============================] - 0s 73us/step - loss: 0.0632 - acc: 0.9806
Epoch 1793/2000
103/103 [==============================] - 0s 60us/step - loss: 0.0283 - acc: 0.9903
Epoch 1794/2000
103/103 [==============================] - 0s 49us/step - loss: 0.0619 - acc: 0.9709
Epoch 1795/2000
103/103 [==============================] - 0s 69us/step - loss: 0.0664 - acc: 0.9806
Epoch 1796/2000
103/103 [==============================] - 0s 106us/step - loss: 0.0520 - acc: 0.9806
Epoch 1797/2000
103/103 [==============================] - 0s 74us/step - loss: 0.0407 - acc: 0.9903
Epoch

103/103 [==============================] - 0s 59us/step - loss: 0.0664 - acc: 0.9612
Epoch 1870/2000
103/103 [==============================] - 0s 59us/step - loss: 0.0810 - acc: 0.9709
Epoch 1871/2000
103/103 [==============================] - 0s 62us/step - loss: 0.0191 - acc: 1.0000
Epoch 1872/2000
103/103 [==============================] - 0s 55us/step - loss: 0.0781 - acc: 0.9709
Epoch 1873/2000
103/103 [==============================] - 0s 77us/step - loss: 0.0364 - acc: 0.9903
Epoch 1874/2000
103/103 [==============================] - 0s 52us/step - loss: 0.0279 - acc: 0.9903
Epoch 1875/2000
103/103 [==============================] - 0s 76us/step - loss: 0.0764 - acc: 0.9806
Epoch 1876/2000
103/103 [==============================] - 0s 54us/step - loss: 0.0680 - acc: 0.9806
Epoch 1877/2000
103/103 [==============================] - 0s 72us/step - loss: 0.0169 - acc: 1.0000
Epoch 1878/2000
103/103 [==============================] - 0s 64us/step - loss: 0.0388 - acc: 0.9806
Epoch 

103/103 [==============================] - 0s 56us/step - loss: 0.0300 - acc: 0.9903
Epoch 1951/2000
103/103 [==============================] - 0s 64us/step - loss: 0.0140 - acc: 1.0000
Epoch 1952/2000
103/103 [==============================] - 0s 83us/step - loss: 0.0148 - acc: 1.0000
Epoch 1953/2000
103/103 [==============================] - 0s 67us/step - loss: 0.0157 - acc: 1.0000
Epoch 1954/2000
103/103 [==============================] - 0s 55us/step - loss: 0.0180 - acc: 0.9903
Epoch 1955/2000
103/103 [==============================] - 0s 58us/step - loss: 0.0238 - acc: 0.9903
Epoch 1956/2000
103/103 [==============================] - 0s 53us/step - loss: 0.0260 - acc: 0.9903
Epoch 1957/2000
103/103 [==============================] - 0s 51us/step - loss: 0.0165 - acc: 1.0000
Epoch 1958/2000
103/103 [==============================] - 0s 56us/step - loss: 0.0112 - acc: 1.0000
Epoch 1959/2000
103/103 [==============================] - 0s 55us/step - loss: 0.0253 - acc: 0.9806
Epoch 

In [23]:
model.predict_classes(X)

array([0, 2, 1, 0, 2, 1, 0, 1, 0, 0, 0, 0, 1, 2, 2, 0, 0, 2, 0, 0, 2, 2,
       2, 2, 0, 2, 0, 1, 0, 2, 1, 1, 0, 1, 0, 1, 0, 2, 0, 1, 1, 0, 0, 1,
       0, 2, 1, 2, 0, 2, 1, 0, 2, 0, 0, 0])

In [24]:
y

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0

# It Works! Let's try it out and use our original Data as Test

Finally, after giving it a once over, it looks like the data mostly matches. Let's ensure it's a perfect fit by using keras' `evaluate` to run a test.

In [27]:
model.test_on_batch(X, y)

[0.010690006, 1.0]

# Hooray! Let's save it now!

We use keras' great built in function, `model.save` to save it to disk. 

In our program for inference, we'll use `keras.models.load_model(filepath)` to load it for deteting dab, tpose, or other.

In [28]:
model.save('data/dab-tpose-other.h5')